In the other jupyter notebook I created for myself a demographic data shapefile at the block level, which is stored in `C:/Users/madie/OneDrive/data/ipums/VA_block_2010_pop.zip`. What I need to now do is aggregate that block-level data into the precinct shapefiles that I got from the VEST team's dataverse. 

In [3]:
import geopandas
import pandas as pd
import maup
import warnings

# turn off annoying geoseries isna warnings
warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)

def aggregate(blocks_f_name, precincts_f_name, year):
    data_dir = "C:/Users/madie/OneDrive/data/"
    zip_data_dir = "zip://" + data_dir
    blocks = geopandas.read_file(zip_data_dir + blocks_f_name)
    precincts = geopandas.read_file(zip_data_dir + precincts_f_name)
    
    year_pref = "G" + str(year) + "HOD"
    # rename the useful cols
    precincts = precincts.rename(columns = {year_pref + "DEM" : "demvote", year_pref + "REP": "repvote"})
    # drop the useless cols
    precincts = precincts.drop([year_pref + "IND", year_pref + "LIB", 
                                      year_pref + "GRE", year_pref + "WRI"], axis = 1)
    
    # change crs of shp files to be projected (using VA north)
    blocks = blocks.to_crs(epsg=2283)
    precincts = precincts.to_crs(epsg=2283)

    # remove any bowties (little imperfections in the polygons)
    blocks.geometry = blocks.buffer(0)
    precincts.geometry = precincts.buffer(0)

    # cols to aggregate from blocks to precincts
    variables = ['pop', 'vap', 'TotPop', 'BlackPop', 'HispPop', 'VAP_1', 'BlackVAP', 'HispVAP']
    with maup.progress():
        assignment = maup.assign(blocks, precincts)
    precincts[variables] = blocks[variables].groupby(assignment).sum()
    
    precincts.to_file(data_dir + "pre-redist/VA_HOD_20" + str(year) + "/VA_HOD_20" + str(year) + ".shp")

In [4]:
blocks_f_name = "ipums/VA_block_2010_pop.zip"
years = [15, 17, 19]
for y in years:
    precincts_f_name = "vest/va_20" + str(y) + "_statehouse.zip"
    aggregate(blocks_f_name, precincts_f_name, y)

100%|██████████████████████████████████████████████████████████████████████████████| 2585/2585 [06:52<00:00,  6.27it/s]


KeyError: "['G19HODIND' 'G19HODLIB' 'G19HODGRE' 'G19HODWRI'] not found in axis"

In [5]:
df_19 = geopandas.read_file("zip://C:/Users/madie/OneDrive/data/vest/va_2019_statehouse.zip")

In [6]:
df_19.head()

,COUNTYFP,LOCALITY,VTDST,PRECINCT,HOD_DIST,geometry
0,001,Accomack County,000101,Chincoteague,100,"POLYGON Z ((-75.40929 37.82092 0.00000, -75.40..."
1,001,Accomack County,000201,Atlantic,100,"POLYGON Z ((-75.41651 37.93483 0.00000, -75.41..."
2,001,Accomack County,000202,Greenbackville,100,"POLYGON Z ((-75.41651 37.93483 0.00000, -75.41..."
3,001,Accomack County,000301,New Church,100,"POLYGON Z ((-75.49376 37.93167 0.00000, -75.49..."
4,001,Accomack County,000401,Bloxom,100,"POLYGON Z ((-75.59939 37.87382 0.00000, -75.59..."
